## Compare two object detection models

In [ ]:
# Import all the required libraries
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Import object_detections dataset
ground_truth = pd.read_csv('data/gt_boxes.csv')
inference_1 = pd.read_csv('data/model_1.csv')
inference_2 = pd.read_csv('data/model_2.csv')

In [ ]:
# Define some possible metric(s) to compare two multi-class object detection models
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [ ]:
# Initialize the IOU, Accuracy and Weighted Accuracy to zero for both inferences.
stats = pd.DataFrame(columns=['iou_1', 'iou_2', 'acc_1', 'acc_2', 'acc_w_1', 'acc_w_2'])

length = len(ground_truth)
for idx in range(length):
    data = {'iou_1': 0, 'iou_2': 0, 'acc_1': 0, 'acc_2': 0, 'acc_w_1': 0, 'acc_w_2': 0}
    image_id = ground_truth.loc[idx]['image_id']
    label = ground_truth.loc[idx]['label']
    ground_truth_bbox = [ground_truth.loc[idx]['xmin'], ground_truth.loc[idx]['ymin'], ground_truth.loc[idx]['xmax'], ground_truth.loc[idx]['ymax']]
    # Corresponding image_id and label
    inference_1_match = inference_1[(inference_1['image_id'] == image_id) & (inference_1['label'] == label)]
    inference_2_match = inference_2[(inference_1['image_id'] == image_id) & (inference_2['label'] == label)]
    
    if not inference_1_match.empty:
        # IOU
        inferences_1_bbox = [inference_1_match.loc[idx]['xmin'], inference_1_match.loc[idx]['ymin'], inference_1_match.loc[idx]['xmax'], inference_1_match.loc[idx]['ymax']]
        data['iou_1'] = bb_intersection_over_union(ground_truth_bbox, inferences_1_bbox)
        # Accuracy
        data['acc_1'] = 1 if ground_truth.loc[idx]['label'] == inference_1_match.loc[idx]['label'] else 0
        # Weighted accuracy
        data['acc_w_1'] = data['acc_1'] * inference_1_match.loc[idx]['score']
        
    if not inference_2_match.empty:
        # IOU
        inferences_2_bbox = [inference_2_match.loc[idx]['xmin'], inference_2_match.loc[idx]['ymin'], inference_2_match.loc[idx]['xmax'], inference_2_match.loc[idx]['ymax']]
        data['iou_2'] = bb_intersection_over_union(ground_truth_bbox, inferences_2_bbox)
        # Accuracy
        data['acc_2'] = 1 if ground_truth.loc[idx]['label'] == inference_2_match.loc[idx]['label'] else 0
        # Weighted accuracy
        data['acc_w_2'] = data['acc_2'] * inference_2_match.loc[idx]['score']
        
    # Append
    stats = stats.append(data, ignore_index=True)
    
stats

In [ ]:
# printing two models
print('Inference 1:')
print(f"IOU: {float(stats[['iou_1']].mean()):.2f}")
print(f"Accuracy: {(float(stats['acc_1'].count()) / length):.2f}")
print(f"Weighted Accuracy: {float(stats[['acc_w_1']].mean()):.2f}")

print('')
print('Inference 2:')
print(f"IOU: {float(stats[['iou_2']].mean()):.2f}")
print(f"Accuracy: {(float(stats['acc_2'].count()) / length):.2f}")
print(f"Weighted Accuracy: {float(stats[['acc_w_2']].mean()):.2f}")